# Interoperability
Many customers need Databricks and Snowflake to work seamlessly together. We will be covering two different aspects of interoperability:
- **Snowflake** `reads` from Databricks
- **Databricks** `reads` from Snowflake*

_Note: Databricks provides multiple ways to access data residing in Snowflake. Some options are best suited for ad-hoc or lightweight processing, some are better suited for strict governance requirements_

In [0]:
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("iceberg_catalog", "")
dbutils.widgets.text("iceberg_schema", "")
dbutils.widgets.text("schema", "")

catalog = dbutils.widgets.get("catalog")
iceberg_catalog = dbutils.widgets.get("iceberg_catalog")
iceberg_schema = dbutils.widgets.get("iceberg_schema")
schema = dbutils.widgets.get("schema")

## Databricks Reads Snowflake Tables
<img src = "./setup/databricks_reads_snowflake_arch.png" width="800">


In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS `iceberg_external_loaction_theme_park`
URL 's3://themeparkiceberg'
WITH (CREDENTIAL databrics_iceberg_role) --you need to create a role in aws
COMMENT 'External location for iceberg data';

In [0]:
%sql
SHOW EXTERNAL LOCATIONS

name,url,comment
00-lk2311-cuj-fat-sandboxfeaturetest-externallocation,s3://00-lk2311-cuj-fat-sandboxfeaturetest-bucket/,This is the external location feature that will access the S3 bucket
_advait_boehringer_image_recog_poc,s3://one-env-uc-external-location/_advait_boehringer_image_recog_poc,null
_advait_dasco_boehringer,s3://one-env-uc-external-location/advaitgodbole,null
_satishsathiya_hmsfed,s3://one-env-uc-external-location/_satishsathiya_hmsfed,external location for hms fed testing
a9o-extlocation01,s3://a9o-extlocation01-s3/,null
aaboode_ext_loc,s3://one-env-uc-external-location/aaboode,
aaugustyniak-external-location,s3://one-env-uc-external-location/aaugustyniak,Testing Volume
abdi_dlt_stream,s3://one-env-uc-external-location/abdi_aon_amazon,null
abe-demo-ext-loc,s3://one-env-uc-external-location/abe,null
abhi-one-env-ext,s3://one-env-uc-external-location/abhi,null


## Create Iceberg Tables in Snowflake

For customers who are looking to have their Snowflake Iceberg tables availible in Databrick:
- Create an [External Iceberg Table](https://docs.snowflake.com/en/user-guide/tutorials/create-your-first-iceberg-table#create-an-external-volume) in Snowflake. The Volume in Snowflake should point to the location we designated above.
- Hydrate the table with data (eg CTAS, Insert, etc).
![](./setup/create_iceberg_snowflake_worksheet.png)

# Create Connection to Horizon Catalog
We will create a [catalog connection](https://e2-demo-field-eng.cloud.databricks.com/explore/locations?o=1444828305810485) similar to a foriegn database connection. However we will specify:
- The external location we created above
- An external location to store metadata

In [0]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import re

with open("PATH/TO/YOUR/PRIVATE_KEY", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )
private_key_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
).decode("utf-8")
private_key_hex64 = re.sub(r"-----.*-----|\n", "", private_key_pem)
# print(private_key_hex64)

In [0]:
display(spark.read.table(f'{iceberg_catalog}.{iceberg_schema}.theme_park_visitor_iceberg'))

CUSTOMERID,PASTSKIPPASS,VISITFREQUENCY,TOTALSPENDINGUSD,AVGQUEUETIMEMIN,AVGRIDETIMEMIN,PREFERREDRIDE,GROUPTYPE,VISITTIMING,TICKETTYPE,AGEGROUP,LASTVISITDATE
1,0,9.86849973612838700,935.18,53.4,3.5,Transformers: The Ride,Family (2+2),Peak (Summer),One-Day,45-54,2024-10-18
2,0,2.85282410046208100,1053.48,23.9,5.6,The Incredible Hulk Coaster,Family (2 adults),Off-Peak (Weekend),One-Day,55+,2024-10-13
3,0,5.00418779832708800,1053.65,103.8,7.0,The Incredible Hulk Coaster,Family (2 adults),Peak (Holiday),VIP Experience,18-24,2024-10-12
4,1,46.44851673084228600,949.19,55.9,6.4,Hagrid's Magical Creatures Motorbike Adventure,Couple,Off-Peak (Weekend),Annual Pass,18-24,2024-08-14
5,1,9.76785602065858600,790.06,49.6,4.8,Transformers: The Ride,Family (2 adults),Peak (Summer),One-Day,55+,2024-09-08
6,0,17.32772917802489600,562.49,69.1,4.1,Revenge of the Mummy,Couple,Peak (Holiday),Annual Pass,45-54,2024-12-14
7,1,8.96317421616424000,618.41,52.5,5.2,Jurassic Park River Adventure,Solo,Peak (Holiday),Annual Pass,45-54,2024-08-09
8,0,6.71497473934111600,907.71,55.9,4.8,The Incredible Hulk Coaster,Large Group (5+),Off-Peak (Weekend),Multi-Day,18-24,2025-02-21
9,1,2.74431833462668200,1073.43,35.5,6.2,Spider-Man Ride,Family (2+2),Peak (Holiday),Annual Pass,35-44,2025-04-08
10,1,15.56773218101206300,575.23,78.0,6.0,Harry Potter and the Escape,Family (2+2),Off-Peak (Weekday),Annual Pass,45-54,2025-04-18


## Snowflake Reads Databricks Tables
Connect to Unity Catalog's Iceberg REST APIs from Snowflake to read a single source data file as Iceberg.
- Write a table to UC
- Generate Iceberg Metadata
- Enable Snowflake & Databricks Catalog Integration
<img src="./setup/snowflake_reads_databricks_arch.png" width="800">

### Documentation
We will be following this awesome blog, [here](https://www.databricks.com/blog/read-unity-catalog-tables-in-snowflake)!

In [0]:
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema", "")

catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
%sql
CREATE TABLE sb.theme_park.visitors_metric_view_snowflake_V2
as
SELECT
 date_format(`Month`, 'yyyy-MM') as `Month`,
 MEASURE(TotalVisits) as `Total_Visits`,
 MEASURE(TotalRevenue) as `Total_Revenue`,
 MEASURE(TotalSpendingPerCustomer) as `Avg_Spending_per_CX`
FROM sb.theme_park.visitors_metric_view -- hear is the metric view
GROUP BY ALL
ORDER BY 1 ASC

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- iceberg v2 does not support deletion vectors, v3 does
REORG TABLE sb.theme_park.visitors_metric_view_snowflake_V2 APPLY (UPGRADE UNIFORM(ICEBERG_COMPAT_VERSION=2));

path,metrics
s3://databricks-e2demofieldengwest/b169b504-4c54-49f2-bc3a-adf4b128f36d/tables/e9d49fac-dd75-4a94-9c74-73d9fad52cf4,"List(1, 1, List(2304, 2304, 2304.0, 1, 2304), List(1648, 1648, 1648.0, 1, 1648), 0, null, null, 0, 1, 1, 0, true, 0, 0, 1750387413573, 1750387415085, 8, 1, null, List(0, 0), null, 4, 4, 194, 0, null)"


In [0]:
%sql
ALTER TABLE sb.theme_park.visitors_metric_view_snowflake_V2 SET TBLPROPERTIES(
  'delta.columnMapping.mode' = 'name',
  'delta.enableIcebergCompatV2' = 'true',
  'delta.universalFormat.enabledFormats' = 'iceberg');

# Authentication
It is recommended to use a service principal for both development and production workloads.

What you'll need:
- Create Service Principal in Workspace Admin Settings
- Save `client id` & `secret`

# Databricks Table in Snowflake
Snowflake Horizon catalog is able to connect to UC's Iceberg REST Catalog. After setting up the catalog integration, we will use Snowflake's [vended credentials](https://docs.snowflake.com/en/user-guide/tables-iceberg-configure-catalog-integration-vended-credentials) for Iceberg to simplify access to underlying storage.
![](./setup/snowflake_read_databricks_worksheet)